In [1]:
from selenium import webdriver
import time
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
from urllib.parse import urljoin
import re
import time
import threading

In [2]:
driver =  webdriver.Chrome(executable_path = 'D:\workspace\chromedriver.exe' )

driver.get('https://ev.or.kr/evmonitor')
#time.sleep(5)

In [3]:
import time
def thread_run():
    print('====',time.ctime(),'====')
    sid = [] #충전소ID
    name =[]  #충전소_이름
    c_name=[] #충전기_충전소이름
    add = []  #주소
    u_time =[]  #사용가능시간
    typ = []  # 구분
    agency = [] # 운영기관
    phone = [] # 연락처
    fee = [] # 충전요금
    amount = [] # 급속충전량
    etc = [] # 참고사항
    using = [] #사용현황
    charge = [] #충전기타입
    city = [] #시
    gu = [] #구
    
    for k in range(2,19):
        driver.find_element_by_css_selector('#F_SI_DO1')
        driver.implicitly_wait(5)
        driver.find_element_by_css_selector('#F_SI_DO1 > option:nth-child(%s)'%k).click()
        #time.sleep(3)
        driver.find_element_by_css_selector('#statList')
        driver.implicitly_wait(30)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        a = soup.find_all('div',class_='fw_path')
        for link in a:
            temp = str(link['sid'])
            sid.append(temp)
    print('id_finish')

    for i in sid[0:10]:
        url_base = 'https://www.ev.or.kr/portal/monitor/evminfo?sid='
        url = url_base+str(i)
        html = urlopen(url)
        soup = BeautifulSoup(html, 'html.parser')

        for tmp_using in soup.find_all(class_='table_01'):
            #if len(tmp_using)<=5:
            n_tmp = soup.find_all('h2')
            name.append(n_tmp[0].get_text())

            t_tmp = soup.find_all('em')
            u_time.append(t_tmp[0].get_text())

            info_tmp = soup.find_all(class_='table_02')
            dinfo = info_tmp[0].find_all('td')
            add.append(dinfo[0].get_text())
            agency.append(dinfo[1].get_text())
            phone.append(dinfo[2].get_text())

            tmp_fee = dinfo[3].get_text()
            tmp_fee = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_fee)
            fee.append(tmp_fee)


            tmp_etc = dinfo[5].get_text()
            tmp_etc = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_etc)
            etc.append(tmp_etc)

            city_tmp = dinfo[0].get_text()[0:2]
            gu_tmp = dinfo[0].get_text().split(' ')
            gu_tmp = gu_tmp[1]
            gu.append(gu_tmp)
            city.append(city_tmp)

            if len(tmp_using)<=5:

                n_tmp = soup.find_all('h2')
                c_name.append(n_tmp[0].get_text())

                ty_tmp = soup.find_all('dd')
                typ.append(ty_tmp[0].get_text())

                tmp_amount = dinfo[4].get_text()

                tmp_amount = re.findall('\d+', tmp_amount) #숫자만 추출
                amount.append(tmp_amount)

                for u_tmp in soup.find_all(class_='td3'):
                    tmp = re.sub('\n', '', u_tmp.get_text())
                    using.append(tmp)


                td2 = soup.find_all(class_='td2')
                for te in td2[0:]:
                    test=[]
                    for a in te.find_all('span'):
                        if a.get_text()=='사용가능':
                            break
                        elif a.get_text()=='사용중':
                            break
                        elif a.get_text()=='사용불가':
                            break
                        elif a.get_text()==' ':
                            break
                        elif len(a.get_text())<6:
                            tmp=[]
                            tmp.append(a.get_text())
                                #print(tmp)
                            test+=tmp
                    test= str("+".join(test))
                    charge.append(test)
                    if test!=None:
                        break
            elif len(tmp_using)<=7:
                tmp_using = soup.find_all(class_='ev_type t03')
                        #print(tmp_using)
                for te in tmp_using[0:]:
                            #print(te)
                    for a in te.find_all('span'):
                        if a.get_text()==' ':
                            break
                        elif len(a.get_text())<6:
                            charge.append(a.get_text())

                            n_tmp = soup.find_all('h2')
                            c_name.append(n_tmp[0].get_text())

                            ty_tmp = soup.find_all('dd')
                            typ.append(ty_tmp[0].get_text())

                            tmp_amount = dinfo[4].get_text()

                            tmp_amount = re.findall('\d+', tmp_amount) #숫자만 추출
                            amount.append(tmp_amount)

                            for u_tmp in soup.find_all(class_='td3'):
                                tmp = re.sub('\n', '', u_tmp.get_text())
                                using.append(tmp)
                                break
    print('data_finish')
    print(name)
    print(city, gu)
    threading.Timer(900, thread_run).start()
thread_run()


==== Fri Sep  4 12:02:54 2020 ====
id_finish
data_finish
['DMC 산학협력연구센터', 'Park M (주차복합빌딩)', 'SETEC', 'sh서울주택도시공사', '가로공원로 지하공영주차장', '가양레포츠센터', '가오리 공영주차장', '강남구청 삼성로 별관 부설 주차장', '강남치매지원센터 공영노외주차장', '강동구청']
['서울', '서울', '서울', '서울', '서울', '서울', '서울', '서울', '서울', '서울'] ['마포구', '마포구', '강남구', '강남구', '강서구', '강서구', '강북구', '강남구', '강남구', '강동구']
==== Fri Sep  4 12:03:25 2020 ====
id_finish
data_finish
['DMC 산학협력연구센터', 'Park M (주차복합빌딩)', 'SETEC', 'sh서울주택도시공사', '가로공원로 지하공영주차장', '가양레포츠센터', '가오리 공영주차장', '강남구청 삼성로 별관 부설 주차장', '강남치매지원센터 공영노외주차장', '강동구청']
['서울', '서울', '서울', '서울', '서울', '서울', '서울', '서울', '서울', '서울'] ['마포구', '마포구', '강남구', '강남구', '강서구', '강서구', '강북구', '강남구', '강남구', '강동구']
